In [1]:
import cv2
import numpy as np

In [8]:
img = cv2.imread('car.jpg', 0)
fil_img = cv2.Laplacian(img, cv2.CV_16S, 3)

cv2.imwrite('laplace_img.jpg', fil_img)
cv2.imshow('image',fil_img)
cv2.waitKey(0) # Wait indefinitely for a keystroke
cv2.destroyAllWindows()

In [2]:
#Task 1
#1) Use of Second Derivative for Image Enhancement: The Laplacian

#convolution funtion
def con2D(img, kernel, padding=1):
    #flipping the kernal because we getting the convolution not correlation
    kernel = np.flipud(np.fliplr(kernel))

    # extracting the dimensions from input and kernal
    Kernal_rows = kernel.shape[0]
    Kernal_cols = kernel.shape[1]
    Img_rows = img.shape[0]
    Img_cols = img.shape[1]

    # calculating the shape of convolved output
    output_rows = ((Img_rows - Kernal_rows + 2 * padding) ) + 1
    output_cols = ((Img_cols - Kernal_cols + 2 * padding) ) + 1
    output = np.zeros((output_rows, output_cols),  np.uint8)

  
    # creating matrix with zeros with padding 
    padded_img = np.zeros((Img_rows + padding*2, Img_cols + padding*2)) 
    
    #updating the matrix with the given image
    padded_img[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = img  
   

    # Iterate through image
    for y in range(Img_cols):            
        for x in range(Img_rows):
            
            #calculating the value of kernal   
            k = (kernel *  padded_img[x: x +  Kernal_rows, y: y + Kernal_cols]).sum()    
    
            #if the k value is minus value, the value is replaced with zero(images doesn't have (-) pixel values)
            #if k value is greater than 255, then it is replaced with 255
            if k >= 255:
                output[x,y] = 255
                
            elif k <= 0:
                output[x,y] = 0
                
            else:
                output[x,y] = k
            

    return output

In [6]:
#● Write a program to implement “The Laplacian” and note the effects on the given image.
def laplacian(img):
    
    #mask for laplacian second derivative enhancement
    laplacian_mask = np.array([[0,1,0], [1,-4,1], [0,1,0]])
    
    #getting the convolution
    new_img = con2D(img, laplacian_mask, padding = 1)
    
    return new_img

In [17]:
laplacian_filter = laplacian(img)
print(laplacian_filter)
cv2.imwrite('car_gray.jpg', img)
cv2.imwrite('laplacian_car.jpg',laplacian_filter)
cv2.imwrite('sharpened_car.jpg',cv2.subtract(img, laplacian_filter))
k = np.min(laplacian_filter)

laplacian_filter = laplacian_filter - k

print(255.0/np.max(laplacian_filter))
laplacian_filter = laplacian_filter * (255.0/np.max(laplacian_filter))
cv2.imwrite('ad.jpg',laplacian_filter )


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 2 ... 0 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 0]]
1.0


True

In [120]:
#● Write a program to implement “Robert Cross Gradient Operator” and observe the hanges in the image.

def RCG(img):
    
    #kernal to do roberts cross gradient operation
    roberts_cross_main_diagonal = np.array( [[ 0, 0, 0 ],
                                             [ 0, -1, 0 ],
                                             [ 0, 0, 1 ]] )

    roberts_cross_left_diagonal = np.array( [[ 0, 0, 0 ],
                                             [ 0, 0, -1 ],
                                             [ 0, 1, 0 ]] )
    
    #getting the convolutions separately
    con_main_diagonal = con2D(img, roberts_cross_main_diagonal, padding=1)
    con_left_diagonal = con2D(img, roberts_cross_left_diagonal, padding=1)
    
    #adding the abosolute values from the convolved results
    output = cv2.add(np.absolute(con_main_diagonal), np.absolute(con_left_diagonal)) 
 
    
    return output

In [13]:
robert_filter = RCG(img)
cv2.imwrite('robert_filter_own1.jpg',robert_filter)


True

In [60]:
#● Write a program to implement “Sobel Operators” and observe the changes on the image
def Sobel(img):
    
    #kernals for sobal operation
    sobel_x = np.array( [[-1,-2, -1],
                         [ 0, 0, 0 ],
                         [ 1, 2, 1 ]] )
    
    sobel_y = np.array( [[ -1, 0, 1 ],
                         [ -2, 0, 2 ],
                         [ -1, 0, 1 ]] )
     
    #getting the convolution
    con_x = con2D(img, sobel_x, padding=1)
    con_y = con2D(img, sobel_y, padding=1)
    
    #adding the absolute values from the convolved results
    output = cv2.add(np.absolute(con_x), np.absolute(con_y))
   
    
    return output

In [61]:
sobel_filter = Sobel(img)
cv2.imwrite('sobel_filter_own12.jpg',sobel_filter)


True

In [7]:
img2 = cv2.imread('eagle.jpg', 0)
cv2.imwrite('eagle_gray.jpg',img2)
img_laplacian = laplacian(img2)

cv2.imwrite('laplacian_eagle.jpg',img_laplacian)

img_sharp = cv2.subtract(img2, img_laplacian)

cv2.imwrite('shapned_eagle.jpg',img_sharp)

True

In [26]:
#Averaging funtion for Task 2

def fil_average(img, mask_size):
    mask = np.ones([mask_size, mask_size], dtype = int)
    mask = mask / (mask_size ** 2)
    
    output = con2D(img,  mask, padding=2)
    
    return output

In [87]:
sobel_eagle = Sobel(img2)
smooth_sobel = fil_average(sobel_eagle, 5)
cv2.imwrite('smoothsobel_eagle.jpg',smooth_sobel)
cv2.imwrite('sobel_eagle.jpg',sobel_eagle)

True

In [90]:
dst = cv2.bitwise_and(img_laplacian, smooth_sobel)
cv2.imwrite('maskedwithsobel.jpg',dst)
print(dst)

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [94]:
add_img2 = cv2.add(dst, img2)
cv2.imwrite('added_new.jpg',add_img2)
print(add_img2)

[[188 189 189 ... 185 185 185]
 [189 190 191 ... 186 187 186]
 [190 191 191 ... 188 188 188]
 ...
 [100 101 101 ...  89  90  88]
 [101 101 101 ...  89  89  88]
 [102 101 100 ...  89  89  88]]


In [95]:
add_img = cv2.subtract(img2, dst)
cv2.imwrite('added.jpg',add_img)

True

In [75]:
sobel_car = Sobel(img)
smooth_sobel_car = fil_average(sobel_car, 5)
cv2.imwrite('smoothsobel_car.jpg',smooth_sobel_car)
cv2.imwrite('sobel_car.jpg',sobel_car)

True

In [83]:
dst10 = cv2.bitwise_and(laplacian_filter, smooth_sobel_car)
cv2.imwrite('multiplt_car.jpg',dst10)

True

In [86]:
add_img23 = cv2.add(img, dst10)
cv2.imwrite('added_new_car.jpg',add_img23)


True

In [104]:
#funtion to do the power law transformation
def powerTrans(I, G):
    
    rows = I.shape[0]
    cols = I.shape[1]
    
    gamma_corrected_img = np.array(255*(I / 255) ** G, dtype = 'uint8')
    
    return gamma_corrected_img

In [121]:
def main():
    img1 = cv2.imread('car.jpg', 0)
    
    #Task 1
    #1) Use of Second Derivative for Image Enhancement: The Laplacian
    #● Write a program to implement “The Laplacian” and note the effects on the given image
    
    laplacian_filter = laplacian(img1)
    cv2.imwrite('laplacian_car.jpg',laplacian_filter)
    
    #2) Use of First Derivative for Image Enhancement: The Gradient
    #● Write a program to implement “Robert Cross Gradient Operator” and observe the changes in the image.
    
    robert_filter = RCG(img1)
    cv2.imwrite('robert_cross_gradient_car.jpg',robert_filter)
    
    #● Write a program to implement “Sobel Operators” and observe the changes on the image.
    
    sobel_car = Sobel(img1)
    cv2.imwrite('sobel_car.jpg',sobel_car)
    
    #Task 2
    #1. Read a given image.
    img = cv2.imread('eagle.jpg', 0)
    
    #2. Take the Laplacian of the image using the Second Derivative.
    
    laplacian_filter = laplacian(img)
    cv2.imwrite('2_2.jpg',laplacian_filter)
    print('2.2) laplacian filtered eagle image is created')
    print('\n')
    
    #3. Sharpen image by adding original image and laplacian image.
    #since the value of the middle cell of the mask is minus we have to subtract the laplace image from original image
    
    sharpened_img = cv2.subtract(img, laplacian_filter)
    cv2.imwrite('2_3.jpg', sharpened_img)
    print('2.3) Sharpened image is created by subtracting laplace image and original image')
    print('\n')
    
    #4. Take the Sobel gradient of the image using the First Derivative.
    sobel_img = Sobel(img)
    cv2.imwrite('2_4.jpg',sobel_img)
    print('2.4) Sobel gradient eagle image is created')
    print('\n')
    
    #5. Smooth Sobel image using 5× 5averaging filter.
    smooth_sobel_img = fil_average(sobel_img, 5)
    cv2.imwrite('2_5.jpg', smooth_sobel_img)
    print('2.5) Smoothed sobel eagle image is created')
    print('\n')
    
    #6. Mask the laplacian image using smooth Sobel image.
    masked_img = cv2.bitwise_and(smooth_sobel_img, laplacian_filter)
    cv2.imwrite('2_6.jpg',masked_img)
    print('2.6) Masked eagle image is created')
    print('\n')
    
    #7. Add product image with the original image.
    #Because of the laplacian mask in this also subtraction should be done
    
    added_img = cv2.subtract(img, masked_img)
    cv2.imwrite('2_7.jpg', added_img)
    print('2.7) Subtracted eagle image is created')
    print('\n')
    
    #8. For better result apply power_law transformation
    final = powerTrans(added_img, 0.6)
    cv2.imwrite('2_8.jpg', final)
    print('2.8) Power transformed final eagle image is created')
    print('\n')
    

if __name__ == "__main__":
    main()
    

2.2) laplacian filtered eagle image is created


2.3) Sharpened image is created by subtracting laplace image and original image


2.4) Sobel gradient eagle image is created


2.5) Smoothed sobel eagle image is created


2.6) Masked eagle image is created


2.7) Subtracted eagle image is created


2.8) Power transformed final eagle image is created


